In [1]:
## Import packages

import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame

In [2]:
## find file path

%cd /Users/tianqi/Downloads/Capstone Data

/Users/tianqi/Downloads/Capstone Data


In [3]:
## read train dataset as data

data = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/train.csv')

/Users/tianqi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## read other datasets

holiday = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/holidays_events.csv')
items = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/items.csv')
oil = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/oil.csv')
stores = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/stores.csv')
transactions = pd.read_csv('/Users/tianqi/Downloads/Capstone Data/transactions.csv')

In [5]:
## Convert data type of datasets
data['date'] =  pd.to_datetime(data['date'], infer_datetime_format=True)
transactions['date'] =  pd.to_datetime(transactions['date'], infer_datetime_format=True)
oil['date'] =  pd.to_datetime(oil['date'], infer_datetime_format=True)
holiday['date'] =  pd.to_datetime(holiday['date'], infer_datetime_format=True)

## Merge datasets

Merge train and stores as "merge01"

In [6]:
### train and stores have different number of rows, so we use left join

merge01=pd.merge(data,stores,on='store_nbr',how="left")

Merge holiday as merge02

In [7]:
merge02 = pd.merge(merge01, holiday[['date', 'locale_name', 'type', 'transferred']], 
                  left_on = ['date', 'state'], 
                  right_on = ['date', 'locale_name'], how = 'left')

del stores
del merge01

In [8]:
merge02 = merge02.drop(columns = 'locale_name')
merge02.columns = ['id', 'date', 'store_nbr', 'item_nbr', 
                   'unit_sales', 'onpromotion', 'city', 
                   'state', 'store_type', 'store_cluster', 
                   'holiday_type', 'transferred']

In [9]:
merge02 = pd.merge(merge02, holiday[holiday['locale_name'] == 'Ecuador']
                   [['date', 'type', 'transferred']], 
                   on = 'date', how = 'left')

In [10]:
merge02.loc[merge02['holiday_type'].isna(), 'holiday_type'] = merge02.loc[merge02['holiday_type'].isna(), 'type']
merge02.loc[merge02['transferred_x'].isna(), 'transferred_x'] = merge02.loc[merge02['transferred_x'].isna(), 'transferred_y']
merge02 = merge02.drop(columns = ['type', 'transferred_y'])
merge02.columns = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 
                   'city', 'state', 'store_type', 'store_cluster', 'holiday_type', 'transferred']
merge02.head()

del holiday

Merge transactions as merge03

In [11]:
merge03 = pd.merge(merge02, transactions, 
                  left_on = ['date', 'store_nbr'],
                   right_on = ['date', 'store_nbr'], how = 'left')
merge03.head()

del merge02
del transactions

Merge items as merge04

In [12]:
merge04=pd.merge(merge03,items,on='item_nbr',how="left")
merge04.head()

del items
del merge03

Merge oil prices as merge05

In [13]:
date_range = pd.date_range(start='1/1/2013', end='8/31/2017', freq='D')
full_oil_price = pd.DataFrame(data = date_range)
full_oil_price.columns = ['date']
full_oil_price['date'] = full_oil_price['date'].apply(str).str.slice(stop = 10)
full_oil_price['date'] =  pd.to_datetime(full_oil_price['date'], infer_datetime_format=True)

In [14]:
full_oil_price = full_oil_price.merge(oil, on = 'date', how = 'left')
full_oil_price.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN


In [15]:
full_oil_price.iloc[0,1] = full_oil_price.iloc[1,1]
for i in range(full_oil_price['dcoilwtico'].size):
    if np.isnan(full_oil_price.iloc[i,1]):
        full_oil_price.iloc[i,1] = full_oil_price.iloc[i-1,1]

In [16]:
merge05 = pd.merge(merge04, full_oil_price, on = 'date', how = 'left')

del merge04
del full_oil_price

merge05.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,store_type,store_cluster,holiday_type,transferred,transactions,family,class,perishable,dcoilwtico
0,0,2013-01-01,25,103665,7.0,NaN,Salinas,Santa Elena,D,1,Holiday,False,770.0,BREAD/BAKERY,2712,1,93.14
1,1,2013-01-01,25,105574,1.0,NaN,Salinas,Santa Elena,D,1,Holiday,False,770.0,GROCERY I,1045,0,93.14
2,2,2013-01-01,25,105575,2.0,NaN,Salinas,Santa Elena,D,1,Holiday,False,770.0,GROCERY I,1045,0,93.14
3,3,2013-01-01,25,108079,1.0,NaN,Salinas,Santa Elena,D,1,Holiday,False,770.0,GROCERY I,1030,0,93.14
4,4,2013-01-01,25,108701,1.0,NaN,Salinas,Santa Elena,D,1,Holiday,False,770.0,DELI,2644,1,93.14


## Fill missing values

In [17]:
df = merge05

In [18]:
df.loc[df['onpromotion'] == '', 'onpromotion'] = 'False'
df['onpromotion'] = df['onpromotion'].apply(bool)
df.loc[df['transferred'].isna(), 'transferred'] = True
df.loc[df['holiday_type'].isna(), 'holiday_type'] = 'Work Day'

## Choose Cities

In [19]:
## 17 clusters in total

len(df['store_cluster'].unique())

17

In [20]:
## 1 cluster may have many cities

cluster14 = df.loc[df['store_cluster'] == 14]
cluster14['city'].unique()

array(['Quito', 'Ambato'], dtype=object)

In [21]:
## 22 cities in total

len(df['city'].unique())

22

In [22]:
## 1 city may have many clusters

city_quito = df.loc[df['city'] == 'Quito']
city_quito['store_cluster'].unique()

array([13,  8,  9,  6, 15, 12, 16,  5, 11, 14])

In [23]:
## count total unit sales by cities

group_by_sum = df.groupby('city').sum()

In [24]:
## sort cities by sum

sort_by_sum = group_by_sum.sort_values('unit_sales',ascending=True)
sort_by_sum.head(10)

,id,store_nbr,item_nbr,unit_sales,onpromotion,store_cluster,transferred,transactions,class,perishable,dcoilwtico
city,,,,,,,,,,,
Puyo,85428049771428,20410786,1021881525503,4.110779e+06,99920.0,6494341,827802.0,7.032789e+08,1795288636,196639,4.210417e+07
Playas,84619819788674,48731130,1334952583070,7.702352e+06,372126.0,4176954,1266602.0,9.586872e+08,2597249434,286587,8.705326e+07
El Carmen,100955643229596,89311464,1560057242844,1.109794e+07,453674.0,4961748,1509569.0,1.443508e+09,3198145627,288673,1.031596e+08
Riobamba,111192119769854,25359026,1707514129407,1.142622e+07,456524.0,12679513,1653426.0,2.499698e+09,3499981755,363035,1.118645e+08
Salinas,121893580457949,51581425,1963853225687,1.147381e+07,526516.0,2063257,1861304.0,2.035205e+09,3996702226,598792,1.306976e+08
Ibarra,118818381409692,29165190,1852048668859,1.148762e+07,480095.0,29165190,1775220.0,2.567407e+09,3728321428,385923,1.211048e+08
Guaranda,115611308398167,35813119,1785644475792,1.309706e+07,452044.0,28273515,1719384.0,2.381919e+09,3616850020,365308,1.171061e+08
Manta,180932850027183,118254625,2369038434837,1.397666e+07,241329.0,28495964,2032403.0,2.756453e+09,4431612453,623866,1.158490e+08
Quevedo,118755375435922,64986735,1854700201835,1.422948e+07,508198.0,5907885,1800767.0,2.022074e+09,3841746672,431023,1.234574e+08


In [25]:
## choose cities
array = ["Puyo", "Playas", "El Carmen", 
         "Riobamba", "Salinas", "Ibarra", 
         "Guaranda", "Manta", "Quevedo", "Libertad"]

myCities = df.loc[df['city'].isin(array)]
myCities.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,store_type,store_cluster,holiday_type,transferred,transactions,family,class,perishable,dcoilwtico
0,0,2013-01-01,25,103665,7.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,BREAD/BAKERY,2712,1,93.14
1,1,2013-01-01,25,105574,1.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,GROCERY I,1045,0,93.14
2,2,2013-01-01,25,105575,2.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,GROCERY I,1045,0,93.14


In [26]:
## covers stores in 4 clusters, 7 states

myCities['store_cluster'].unique()

array([ 1,  7, 15,  3, 10, 13, 11])

In [27]:
myCities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18101977 entries, 0 to 125880297
Data columns (total 17 columns):
id               int64
date             datetime64[ns]
store_nbr        int64
item_nbr         int64
unit_sales       float64
onpromotion      bool
city             object
state            object
store_type       object
store_cluster    int64
holiday_type     object
transferred      bool
transactions     float64
family           object
class            int64
perishable       int64
dcoilwtico       float64
dtypes: bool(2), datetime64[ns](1), float64(3), int64(6), object(5)
memory usage: 2.2+ GB


In [28]:
## Choose family
array = ["EGGS"]
Cities10eggs = myCities.loc[myCities['family'].isin(array)]
Cities10eggs.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,store_type,store_cluster,holiday_type,transferred,transactions,family,class,perishable,dcoilwtico
30,30,2013-01-01,25,158680,1.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,EGGS,2502,1,93.14
56,56,2013-01-01,25,208384,3.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,EGGS,2502,1,93.14
74,74,2013-01-01,25,227111,1.0,True,Salinas,Santa Elena,D,1,Holiday,False,770.0,EGGS,2502,1,93.14


In [29]:
Cities10eggs.to_csv(r'/Users/tianqi/Downloads/Capstone Data/Cities10eggs.csv', header=True, index=False)

In [30]:
Cities10eggs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197557 entries, 30 to 125879633
Data columns (total 17 columns):
id               197557 non-null int64
date             197557 non-null datetime64[ns]
store_nbr        197557 non-null int64
item_nbr         197557 non-null int64
unit_sales       197557 non-null float64
onpromotion      197557 non-null bool
city             197557 non-null object
state            197557 non-null object
store_type       197557 non-null object
store_cluster    197557 non-null int64
holiday_type     197557 non-null object
transferred      197557 non-null bool
transactions     197224 non-null float64
family           197557 non-null object
class            197557 non-null int64
perishable       197557 non-null int64
dcoilwtico       197557 non-null float64
dtypes: bool(2), datetime64[ns](1), float64(3), int64(6), object(5)
memory usage: 24.5+ MB
